<a href="https://colab.research.google.com/github/lyubolp/PythonCourse2022/blob/functional_05/05%20-%20Functional%20Programming/05%20-%20Functional%20Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Функционално програмиране с Python
План на лекцията:
- [ ] Immutability & side-effects
- [X] Функции като обекти
- [X] Декоратори
- [X] Анонимни (lambda) функции
- [ ] Lazy evaluation
- [ ] Generators (Не забравяй за `range`)
- [X] Map
- [X] Filter
- [X] Reduce
- [X] Zip
- [X] List comprehension
- [X] Скорост
- [ ] Примери
- [ ] Задачи

## Immutability & side-effects

## Функции, като обекти


В Python, функциите са т.нар. "first-class objects" - което на български може да се преведе като "първокласни обекти". А ако трябва да го обясним с прости думи, функциите могат да се третират като "нормалните" типове - числа, низове и т.н. Това означава, че можем да ги използваме като променливи - да им даваме имена, да ги подаваме към аргументи на други функции, да ги връщаме като резултат от функции и да ги пазим в колекции.

In [ ]:
def multiply(a, b):
    return a * b

f = multiply

print(f(2, 3))

6


Важно е да направим разлика между `f = multiply` и `f = multiply()`. Макар и разликата да е малка синтактично, поведението на двата реда е различно. Първото извикване **присвоява функцията** към променливата `f`, докато второто **присвоява резултата от извикването** на функцията.  

Когато присвоим цялата функция (може да си го представяме като function pointer в C++), можем да ползваме функцията като нормална променлива:

In [ ]:
def multiply(a, b):
    return a * b

def sum(a, b):
    return a + b


def apply_to_numbers(a, b, f):
    print(f'Applying a function to {a} and {b}')
    return f(a, b)

print(apply_to_numbers(2, 3, sum))
print(apply_to_numbers(2, 3, multiply))

Applying a function to 2 and 3
5
Applying a function to 2 and 3
6


А можем и да връщаме функция като резултат от друга функция:

In [ ]:
def foo():
    print("Foo called")

def bar():
    return foo

f = bar()
f()

Foo called


В тялото на една функция, можем да дефинираме друга функция, която да върнем:

In [ ]:
def foo():
    def bar():
        print("Bar called")
    
    return bar

f = foo()
f() 

Bar called


Това, че функциите са обекти, ни позволява да правим интересни неща - например, можем да направим проста функция, която да използваме за logging.

In [ ]:
def log_and_run(f, args=(), kwargs={}):
    print(f'Calling {f} with {args} and {kwargs}')
    return f(*args, **kwargs)

def add(a, b):
    return a + b

print(log_and_run(add, (2, 3)))

Calling <function add at 0x7f02868be560> with (2, 3) and {}
5


Припомням, че `args` съдържа позиционните аргументи и е от тип `Tuple`, а `kwargs` съдържа keyword аргументите и е от тип `dict`.

Освен като аргументи и резултат, функциите могат да се пазят и в колекции (списъци, речници, т.н.):

In [ ]:
def add(a, b):
    return a + b

def subtract(a, b):
    return a - b

def multiply(a, b):
    return a * b

def divide(a, b):
    return a / b

actions = {
    '+': add,
    '-': subtract,
    '*': multiply,
    '/': divide
}

first = input('Enter a number: ')
operation = input('Enter an operation (+, -, * or /)')
second = input('Enter a number: ')

if operation not in actions:
    print('Operation not supported')
else:
    if not first.isnumeric() or not second.isnumeric():
        print('Input is not a number')
    else:
        first = int(first)
        second = int(second)

        result = actions[operation](first, second)
        print(f'{first} {operation} {second} = {result}')


Enter a number: 2
Enter an operation (+, -, * or /)+
Enter a number: 3
2 + 3 = 5


## Декоратори

Една от по-интересни функционалности на Python са т.нар. декоратори (decorators). Всъщност, ние вече видяхме как се пише декоратор и как работи - нека се върнем на примера от по-горе с `log_and_run`:

In [ ]:
def log_and_run(f, args=(), kwargs={}):
    print(f'Calling {f} with {args} and {kwargs}')
    return f(*args, **kwargs)

def add(a, b):
    return a + b

print(log_and_run(add, (2, 3)))

Calling <function add at 0x7f02868be830> with (2, 3) and {}
5


Това изглежда леко грозно, да трябва да извикваме `log_and_run` вместо функцията, която всъщност искаме да извикаме. Ако добавим функции за изваждане, умножение и деление, всяко извикване ще трябва да преминава през `log_and_run`. 

А няма ли по-лесен начин с който да кажем - нека преди изпълнението на нашата функция `f1`, да се изпълни функция `f` (Както преди `add` се изпълнява `log_and_run`) ?

Преди да разгледаме как точно се прави декоратор функция в Python, нека всъщност разберем какво искаме да постигнем:
1. Искаме да приемем функция (нека я кръстим `g`), която да изпълним
2. Преди да изпълним `g`, искаме да изпълним някакви други действия (нека те са събрани във функцията `f`
3. Искаме да върнем нова функция (нека да я кръстим `h`), която първо да изпълни `f`, а после `g`.

Нека първо се фокусираме върху 1 и 2 :

In [ ]:
def pseudo_decorator(f, g):
    f()
    g()


def print_hello():
    print("Hello")

def print_bye():
    print("bye")


pseudo_decorator(print_hello, print_bye)

Hello
bye


Нека променим поведението на `pseudo_decorator` от това просто да изпълнява `f` и `g` към това да връща нова функция, която да изпълнява `f` и `g`

In [ ]:
def pseudo_decorator(f, g):
    def inner():
        f()
        g()
    return inner


def print_hello():
    print("Hello")

def print_bye():
    print("bye")


f = pseudo_decorator(print_hello, print_bye)
f()

Hello
bye


Почти сме до истинските декоратори. За съжаление, следващата част е възможна заради малко синтактична захар. 

Основната идея е, че искаме да вземем нашата функция `g` и да я заместим с резултата от `pseudo_decorator`, без да се налага реално да извикваме `pseudo_decorator`. С цел опростяване, ще приемем, че `f` е просто принтиране. 

In [ ]:
def print_decorator(g):
    def inner():
        print("Hello")
        g()
    return inner

@print_decorator
def print_bye():
    print("bye")

print_bye()

Hello
bye


С помощта на `@`, последвано от функция, която връща друга функция, ние можем да "декорираме" всяка функция в Python.

Това, което се случва в случая е, че взимаме `print_bye` като първи аргумент на `print_decorator` и изпълняваме `print_decorator`. Тя от своя страна ни връща нова функция, която принтира "Hello" и след това изпълнява подадената от нас функция. 

Обяснено още по-просто: При извикване на `print_bye`, всъщност ще се извика `print_decorator` с аргумент `print_bye`. 

Нека използваме този нов подход върху примера с `log_and_run` (която ще преименуваме само на `log`)

In [ ]:
def log(f):
    def inner(*args, **kwargs):
        print(f'Calling {f} with {args} and {kwargs}')
        return f(*args, **kwargs)
    return inner

@log
def add(a, b):
    return a + b

print(add(2, 3))

Calling <function add at 0x7f0286878dd0> with (2, 3) and {}
5


Сега ако добавим нови методи за изваждане, умножение и деление, можем лесно да се използваме от `log`, без да се налага да променяме начина им на извикване:

In [ ]:
def log(f):
    def inner(*args, **kwargs):
        print(f'Calling {f} with {args} and {kwargs}')
        return f(*args, **kwargs)
    return inner

@log
def add(a, b):
    return a + b

@log 
def subtract(a, b):
    return a - b

@log
def multiply(a, b):
    return a * b

@log
def divide(a, b):
    return a / b

print(add(2, 3))
print(subtract(3, 1))
print(multiply(1.5, b=3))
print(divide(a=12, b=6))

Calling <function add at 0x7f027e9448c0> with (2, 3) and {}
5
Calling <function subtract at 0x7f027e944170> with (3, 1) and {}
2
Calling <function multiply at 0x7f027e95c200> with (1.5,) and {'b': 3}
4.5
Calling <function divide at 0x7f027e95c5f0> with () and {'a': 12, 'b': 6}
2.0


Друг пример за декоратор може да бъде декоратор, който измерва времето за изпълнение на функция:

In [ ]:
import time

def time_it(f):
    def inner(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print(f'{f} took {end - start:.2f} second')
    return inner

@time_it
def slow_function(a):
    return (a ** a) ** a

slow_function(1000)

<function slow_function at 0x7f29e27127a0> took 1.52 second


## Анонимни (lambda) функции

Lambda функции идват от т.нар ламбда смятане (lambda calculus) и изразяват изпълнението на дадени изчисления върху дадени стойности. Няма да навлизаме в математическите доказателства и формализми. Lambda функциите са анонимни функции - т.е. не се дефинират с `def` и без да е необходимо да им се дава име. 

Всяка ламбда функция има две основни части - входни променливи и един израз (expression). В Python, ламбда функциите се дефинират с ключовата дума `lambda`

In [1]:
lambda x: x + 1

<function __main__.<lambda>>

По-горната функция приема един аргумент (на име `x`), и изпълнява израза `x + 1`. Важно е да се отбележи, че една ламбда функция може да съдържа само един израз.

Припомням, че израз в Python е всичко, което съдържа някакъв индетификатор (напр. променлива), литерал (напр. `'hello'`) или оператор (напр. извикване на функция, оператор `[]`)

Или казано по-просто: не може да създадем ламбда, която да присвоява стойност на някаква променлива:

In [4]:
lambda x: a = x

SyntaxError: ignored

Не можем също и да имаме два израза един след друг (Това чисто синтактично няма как да ни се позволи - ламбдите са на един ред, а в Python нямаме символ за край на израз.

Можем да извикваме дадена lambda функция, като просто използваме `()` след нея

In [7]:
(lambda x: print(x))(5)

5


Макар и анонимни, можем да присвоим ламбда функция към дадена променлива

In [8]:
f = lambda x: x + 1

print(f(5))

6


Горното парче код изпълнява следните стъпки:
1. Създава анонимна функция, която приема един аргумент (`x`) и връща стойноста на `x` събрана с 1
2. Присвояваме новосъздадената ни функция към променливата `f`
3. Принтираме резултата от извикването на `f` със стойност 5
    - В тялото на ламбда функцията, стойноста на `x` се замества с 5
    - От там, 5 + 1 = 6

Едно от местата, където често се ползват lambda функции, са вградените функции за сортиране. Както знаем, метода `.sort()` може да приема функция, която да сравнява обектите. 

In [1]:
 l = [5, 2, 7, 3]
 l.sort(key=lambda x: -x)
 print(l)

[7, 5, 3, 2]


С помощта на тази ламбда, можем да сортираме числата в обратен ред. 

Можем да използваме ламбди за да сортираме и по-сложни списъци - например списък от наредени n-торки.

Нека е даден списък, съставен от имена на хора, и тяхната възраст. 

In [4]:
people_data = [('Иван', 22), ('Георги', 71), ('Мария', 35), ('Митко', 51), ('Любо', 35)]
people_data.sort(key=lambda person: (person[1], person[0]))
print(people_data)

[('Иван', 22), ('Любо', 35), ('Мария', 35), ('Митко', 51), ('Георги', 71)]


Тук първо сортираме хората по тяхната възраст, а после лексикографкси, по тяхното име.

## Lazy evaluation

## Generators

## Map



Map е специална функция, която има една-единствена цел - да приложи друга функция към дадена колекция. Нека е даден списък с числа и функция, която приема число, и го умножава по 2.

In [ ]:
numbers = [2, 7, 3, 9, -1, 12]

def multiply(number):
    return number * 2

Ако трябва сами да разпишем map функцията, тя би изглеждала по следния начин:

In [ ]:
def my_map(map_function, collection):
    result = []

    for item in collection:
        result.append(map_function(item))
    
    return result

Нека извикаме нашата map функция, с дефинираните по-горе числа и функция, която да бъде приложена

In [ ]:
print(f'Squared numbers: {my_map(multiply, numbers)}')

[4, 14, 6, 18, -2, 24]


Можем да подадем и функция, която променя типа на обектите - нека разгледаме функция, която приема цяло число от 1 до 7 и връща съответния ден от седмицата (1 - Понеделник, 2 - Вторник, т.н.)

In [ ]:
def number_to_day(number):
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    return days[number - 1]

days = [2, 3, 1, 4, 6]

print(f'Days of the week: {my_map(number_to_day, days)}')

['Tuesday', 'Wednesday', 'Monday', 'Thursday', 'Saturday']


Нека сега разгледаме и вградена функция `map` в Python - тя приема като първи аргумент функцията, която ще бъде приложена, а като втори - колекцията, върху която да се приложи.

In [ ]:
numbers = [1, 2, 3, 4]

def square(number):
    return number ** 2

print(f'Appyling map directly returns: {map(square, numbers)}')

Важно е да се отбележи, че резултатът от map функцията е `map` обект - или поне така изглежда. Всъщност това, което се връща е генератор, който ни дава новите стойности. Съвсем лесно можем да превърнем този генератор в списък, с едно просто извикване на `list()` функцията.


In [ ]:
numbers = [1, 2, 3, 4]

def square(number):
    return number ** 2

print(f'Converting the map object into a list: {list(map(square, numbers))}')

[1, 4, 9, 16]


Освен да подаваме нормална функция, можем да използваме и `lambda`.

In [ ]:
numbers = [1, 2, 3, 4]

mapped_numbers = map(lambda number: number ** 2, numbers)
result = list(mapped_numbers)

print(result)

[1, 4, 9, 16]


Нека усложим нещата - ако се върнем на първоначалната ни дефиниция за `map` (Функция, която прилага друга функция върху колекция), си задаваме въпроса - какво става, ако имаме повече от една колекция, върху която искаме да приложим функция ? Възможно ли е въобще ? 

Отговора е да - `map` приема една или повече колекции. Единствената особенност е, че функцията която подаваме трябва да може да приеме повече от един аргумент. На по-прост език - ако подадем два списъка на `map`, функцията която ще се приложи върху тях трябва да приема два аргумента.

In [ ]:
first_numbers = [1, 2, 3, 4]
second_numbers = [5, 6, 7, 8]

def multiply(a, b):
    return a * b

print(list(map(multiply, first_numbers, second_numbers)))

[5, 12, 21, 32]


Важно е да се отбележи, че резултата от `map` операция винаги е **една** поредица от обекти - дори и да приемем 10 колекции, винаги на изхода ще имаме една. 

**Въпрос:** Как може да "измамим" системата, и да върнем повече от една колекция ?

### Отговор

Няма как да върнем повече от една колекция - това което можем да направим, е да върнем колекция, която съдържа наредени n-торки (tuples).

In [ ]:
first_numbers = [1, 2, 3, 4]
second_numbers = [5, 6, 7, 8]

def return_as_tuple(a, b):
    return a, b

print(list(map(return_as_tuple, first_numbers, second_numbers)))

[(1, 5), (2, 6), (3, 7), (4, 8)]


## Filter


Както името подсказва, `filter` е функция която филтрира елементи, отговарящи на даден критерии.

Нека отново имаме някакви числа, и функция която връща дали дадено число е четно или не.

In [ ]:
numbers = [2, 7, 11, 12]

def is_even(number):
    return number % 2 == 0

Ако трябва сами да разпишем функцията, тя би изглеждала по следния начин:

In [ ]:
def my_filter(filter_fn, collection):
    result = []
    for item in collection:
        if filter_fn(item):
            result.append(item)
    
    return result

In [ ]:
print(my_filter(is_even, numbers))

[2, 12]


Извикването на вградената функция `filter` изглежда по следния начин

In [ ]:
print(filter(is_even, numbers))

Както `map`, така и `filter` връщат генератор обекти - с едно извикване на `list` можем да превърнем резултата в списък

In [ ]:
print(list(filter(is_even, numbers)))

[2, 12]


Може и с `lambda`

In [ ]:
print(list(filter(lambda number: number % 2 == 0, numbers)))

[2, 12]


## Reduce

Тук нещата стават една идея по-сложни. Нека започнем с един пример: трябва да намерим произведението на всички числа в даден списък.

Нека за момент се абстрахираме от програмирането - как бихме намерили произведението на числа 3, 5 и 8 ? Първо бихме умножили 3 и 5, което прави 15. След това, получения резултат от предишната операция (в нашия случай 15) и ще го умножим с 8.

Тази процедура може да бъде използвана за списъци с произволен брой числа - всяко следващо число бива умножено с резултата до момента.

Ако трябва да представим тази идея с код, тя би изглеждала по следния начин:

In [ ]:
def get_product_of_all_nums(nums):
    result = nums[0]

    for num in nums[1:]:
        result = result * num
    
    return result

In [ ]:
numbers = [3, 5, 8]
print(get_product_of_all_nums(numbers))

120


Нека генерализираме нашата функция - вместо умножение, тя да може да работи с подадена функция, приемаща два аргумента - резултата до момента и текущото число

In [ ]:
def my_reduce(function, collection):
    result = collection[0]

    for item in collection[1:]:
        result = function(result, item)
    
    return result

In [ ]:
numbers = [3, 5, 8]
def multiply(result_so_far, current_number):
    return result_so_far * current_number

print(my_reduce(multiply, numbers))

120


Една оптимизация, която можем да направим е следната - вместо за начало да взимаме първия елемент от колекцията, може да приемаме началната стойност като аргумент на функцията

In [ ]:
def my_reduce_with_start(function, collection, start):
    result = start

    for item in collection:
        result = function(result, item)
    
    return result

In [ ]:
numbers = [3, 5, 8]
def multiply(result_so_far, current_number):
    return result_so_far * current_number

print(my_reduce_with_start(multiply, numbers, 1))

120


Ето че стигнахме и до `reduce` - функция, която приема функция и колекция. Функцията бива приложена върху резултата, който имаме до момента и всеки елемент от колекцията.
Важно е да уточним терминологията тук - в някои езици може да срещенете тази функция като `accumulate`, `fold` или пък `foldl`.

Тук е важно да спомена за наличието на т.нар "ляв" и "десен" `reduce`. В други функционални езици (като Haskell), имаме концепцията за ляв и десен `fold` - в каква посока се изпълнява функцията - отляво надясно или отдясно наляво. За някои операции (например умножение) реда на изпълнение няма значение, но например при деление би имало значение дали започваме от ляво или от дясно. Повече информация за fold функцията в другите езици, може да намерите [тук](https://www.wikiwand.com/en/Fold_(higher-order_function)). 

Вградената функция reduce в Python се намира в библиотеката `functools`. За да я използваме, първо трябва да я заредим (`import`)

In [ ]:
from functools import reduce

numbers = [3, 5, 8]

def multiply(result_so_far, current_number):
    return result_so_far * current_number

print(reduce(multiply, numbers, 1))

120


Както може би сте се досетили, тук не получаваме генератор - резултатът от `reduce` е една стойност (в нашия случай това е число).

`reduce`, също както `map` и `filter` работи и с `lambda` функции. Тук е добро място да се отбележи, че първия аргумент, който бива подаден на функцията е стойността до момента (или стойността, която се е **акумулирала** (**accumulate**). 

In [ ]:
numbers = [3, 5, 8]

print(reduce(lambda acc, number: acc * number, numbers))

120


А можем и да подадем начална стойност:

In [ ]:
numbers = [3, 5, 8]
print(reduce(lambda acc, number: acc * number, numbers, 1))
print(reduce(lambda acc, number: acc * number, numbers, 2))

120
240


А сега един по-сложен пример: Нека имаме списък, чийто елементи са списъци с числа. С помощта на `reduce`, подходяща функция и подходяща начална стойност, бихме могли да получим един списък, който съдържа числата, без те да са в отделни списъци.

In [ ]:
numbers_2d = [[1, 2, 3], [4, 5, 6], [7, 8, 9, 10]]

print(reduce(lambda acc, item: acc + item, numbers_2d, []))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


Припомням, че операция събиране (`+`) приложена върху два списъка ни връща конкатенацията на двата списъка. 

В горната операция, ние взехме един списък, и го "изравнихме" (flatten). Операцията "flatten" върху списъци е често срещата в езиците за функционално програмиране, макар и в Python да нямаме готова функция за това. 

## Zip

Нека се върнем за кратко към примера за използването на `map`, при който от два списъка, ние направихме един, който съдържаше елементите на двата, групирани по позицията им:

In [ ]:
first_numbers = [1, 2, 3, 4]
second_numbers = [5, 6, 7, 8]

def return_as_tuple(a, b):
    return a, b

print(list(map(return_as_tuple, first_numbers, second_numbers)))

[(1, 5), (2, 6), (3, 7), (4, 8)]


В Python има готова функция, която прави това обединение за нас - `zip`

In [ ]:
first_numbers = [1, 2, 3, 4]
second_numbers = [5, 6, 7, 8]

print(zip(first_numbers, second_numbers))
print(list(zip(first_numbers, second_numbers)))

[(1, 5), (2, 6), (3, 7), (4, 8)]


Не е нужно списъците да са от еднакви типове елементи.

In [ ]:
names = ['Иван', 'Любо', 'Алекс']
favorite_number = [3, 20, 7]

print(list(zip(names, favorite_number)))

[('Иван', 3), ('Любо', 20), ('Алекс', 7)]


Ако единият списък е по-дълъг от другия, по-дългия списък ще бъде отрязан:

In [ ]:
names = ['Иван', 'Любо', 'Алекс', 'Иво']
favorite_number = [3, 20, 7]

print(list(zip(names, favorite_number)))

[('Иван', 3), ('Любо', 20), ('Алекс', 7)]


In [ ]:
names = ['Иван', 'Любо', 'Алекс']
favorite_number = [3, 20, 7, -1]

print(list(zip(names, favorite_number)))

[('Иван', 3), ('Любо', 20), ('Алекс', 7)]


Или поне това е поведението по подразбиране. `zip` предлага още два метода на работа. С помощта на аргумента `strict=True`, ако подадените списъци са с различен размер, `zip` ще ни хвърли грешка (това е въведено в Python3.10). 

In [ ]:
names = ['Иван', 'Любо', 'Алекс']
favorite_number = [3, 20, 7, -1]

zip(names, favorite_number, strict=True)

TypeError: ignored

А ако искаме да добавим някакви "празни" стойности към по-късия списък, може да използваме 

In [ ]:
from itertools import zip_longest

names = ['Иван', 'Любо', 'Алекс']
favorite_number = [3, 20, 7, -1, -2]

print(list(zip_longest(names, favorite_number, fillvalue='Dummy')))

[('Иван', 3), ('Любо', 20), ('Алекс', 7), ('Dummy', -1), ('Dummy', -2)]


Ако трябва да сме на 100% коректни, `zip` приема каквато и да е колекция:

In [ ]:
numbers = [1, 2, 3]
letters = ['a', 'b', 'c']

print(list(zip(map(lambda x: x ** 2, numbers), letters)))

[(1, 'a'), (4, 'b'), (9, 'c')]


Горният пример, малко по-културно разписан, изглежда по следния начин:

In [ ]:
numbers = [1, 2, 3]
letters = ['a', 'b', 'c']

squared_numbers = map(lambda x: x ** 2, numbers)
zipped_together = zip(squared_numbers, letters)
zipped_together_as_a_list = list(zipped_together)

print(zipped_together_as_a_list)

[(1, 'a'), (4, 'b'), (9, 'c')]


## List comprehension

В Python съществува функционалност, наречена "list comprehension". List comprehension е синтактична конструкция, която ни позволява да създаваме списъци на базата на други списъци. Синтактично, list comprehension следва математическата нотация за дефиниране на множество:

$$ S = \{x^2 \mid x \in [1;100] , x \text{ is even}\} $$

Това математически заклинание ни дава цели числа, по-големи от 5, като върху тях е повдигнато на квадрат.

Можем да изразим това в Python като използваме вече познатите ни `map` и `filter`:


In [ ]:
numbers = range(1, 101)

even_numbers = filter(lambda x: x % 2 == 0, numbers)
squared_numbers = map(lambda x: x ** 2, even_numbers)

print(f'Squared even numbers: {list(squared_numbers)}')

Squared even numbers: [4, 16, 36, 64, 100, 144, 196, 256, 324, 400, 484, 576, 676, 784, 900, 1024, 1156, 1296, 1444, 1600, 1764, 1936, 2116, 2304, 2500, 2704, 2916, 3136, 3364, 3600, 3844, 4096, 4356, 4624, 4900, 5184, 5476, 5776, 6084, 6400, 6724, 7056, 7396, 7744, 8100, 8464, 8836, 9216, 9604, 10000]


В общия случай, синтаксиса на list comprehension е следния: `[f(item) for item in collection if p(item)]` - т.е. кода по-горе би изглеждал по следния начин като list comprehension:

In [ ]:
print(f'Squared even numbers: {[x ** 2 for x in range(1, 101) if x % 2 == 0]}')

Squared even numbers: [4, 16, 36, 64, 100, 144, 196, 256, 324, 400, 484, 576, 676, 784, 900, 1024, 1156, 1296, 1444, 1600, 1764, 1936, 2116, 2304, 2500, 2704, 2916, 3136, 3364, 3600, 3844, 4096, 4356, 4624, 4900, 5184, 5476, 5776, 6084, 6400, 6724, 7056, 7396, 7744, 8100, 8464, 8836, 9216, 9604, 10000]


`if` часта или функцията, която се прилага върху всяка променливата могат да бъдат изпуснати:

In [ ]:
numbers = [1, 2, 3]
print(f'Odd numbers: {[number for number in numbers if number % 2 != 0]}')
print(f'Multiplying all numbers by 2: {[number * 2 for number in numbers]}')

Odd numbers: [1, 3]
Multiplying all numbers by 2: [2, 4, 6]


In [ ]:
def calculate_area_of_rectangle(a, b):
    return a * b

rectangles = [(1, 2), (5, 4), (3, 2.5), (-2, 3)]
areas = [calculate_area_of_rectangle(x, y) for x, y in rectangles if x >= 0 and y >= 0]

print(f'Areas of valid rectangles: {areas}')

Areas of valid rectangles: [2, 20, 7.5]


Може би забелязвате, че когато функцията която изпълняваме не е външна, не използваме `lambda`, а директно пишем операцията която искаме да приложим - това е част от опростяването на синтаксиса на list comprehension-ите. 

Нека имаме дадени числа - искаме да върнем списък, в който за всяко число да пише "Yes" ако се дели на 3 или 5 и "No" в противен случай. Това може да стане лесно, чрез използването на тернарния оператор в Python.

In [ ]:
numbers = [7, 12, 5, 6, 9, 15, 1, 11]

results = ["Yes" if number % 3 == 0 or number % 5 == 0 else "No" for number in numbers]

print(f'Results are: {results}')

Results are: ['No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No']


## Скорост

Едно от най-"шокиращите" неща в Python е скоростта, която получаваме при използването на обикновенни `for` цикли, `map` или list comprehension.

За да демонстрираме разликата, ще създадем функция, която повдига число на квадрат, и ще я приложим върху списък от 40 000 000 числа.

In [ ]:
from time import time

def power_up(a):
    return a ** 2

numbers = list(range(1, 40000000))

for_loop_start = time()

for_loop_result = []
for number in numbers:
    for_loop_result.append(power_up(number))

for_loop_end = time()
print(f'For-looping took {for_loop_end - for_loop_start:.2f} seconds')


map_start = time()

map_result = list(map(power_up, numbers))

map_end = time()
print(f'map() took {map_end - map_start:.2f} seconds')


list_comprehension_start = time()

list_comprehension_result = [power_up(x) for x in numbers]

list_comprehension_end = time()
print(f'List comprehension took {list_comprehension_end - list_comprehension_start:.2f} seconds')

For-looping took 21.92 seconds
map() took 14.21 seconds
List comprehension took 15.65 seconds


Основната разлика във времето идва от факта, че `append()` операцията е бавна. `map` и list comprehension-ите използва хитрини при заделянето на паметта, затова може да се каже че са с около 30% по-бързи. Няма да се впускам повече в темата, за по-любопитните, [тук](https://web.archive.org/web/20190319205826/http://blog.cdleary.com/2010/04/efficiency-of-list-comprehensions/) нещата са обяснение в повече детайли.

На въпросът, кой от трите подхода се използват, моят отговор е следния: list comprehension и `map` дават доста по-добро представяне спрямо `for` loop + `append`, затова и са за предпочитане. Синтаксисът на list comprehension-ите е по-прост спрямо този на `map` (особенно ако трябва и да филтрираме елементите), което води до това, че **list comprehension-ите са за предпочитане**.

## Примери

## Задачи